In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/.s3-optimization-0")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/.s3-optimization-1")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/.s3-optimization-2")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/00000000000000000000.crc")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/00000000000000000000.json")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/_copy_into_log/0.zip")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/_copy_into_log/")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/_delta_log/")

dbutils.fs.rm("dbfs:/user/hive/warehouse/orders/")


True

In [0]:
%sql
drop  table orders

org.apache.spark.sql.catalyst.analysis.NoSuchTableException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`orders` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01
	at org.apache.spark.sql.errors.QueryCompilationErrors$.noSuchTableError(QueryCompilationErrors.scala:1673)
	at org.apache.spark.sql.execution.datasources.v2.DropTableExec.run(DropTableExec.scala:42)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.$anonfun$result$2(V2CommandExec.scala:48)
	at org.apache.spark.sql.execution.SparkPlan.runCommandWithAetherOff(SparkPlan.scala:178)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.datasources.v2.V2Comma

In [0]:
%sql
create table orders(
  order_id string,
  order_date string,
  order_customer_id string,
  order_status string 
)using delta 




In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/orders/_delta_log/_copy_into_log")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2483991118844509>, line 1
----> 1 dbutils.fs.ls("dbfs:/user/hive/warehouse/orders/_delta_log/_copy_into_log")

File /databricks/python_shell/lib/dbruntime/dbutils.py:380, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    378 exc.__context__ = None
    379 exc.__cause__ = None
--> 380 raise exc

ExecutionError: An error occurred while calling o406.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/orders/_delta_log/_copy_into_log
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:126)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:74)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.Da

True

In [0]:
%sql
describe detail orders

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,9beed443-edc5-4a11-8eaf-264666e33c31,spark_catalog.default.orders,null,dbfs:/user/hive/warehouse/orders,2025-07-07T14:26:06.708Z,2025-07-07T14:26:09Z,List(),List(),0,0,Map(),1,2,"List(appendOnly, invariants)",Map()


In [0]:
%fs 

ls /FileStore/orders_csv.csv


path,name,size,modificationTime
dbfs:/FileStore/orders_csv.csv,orders_csv.csv,2999896,1751628328000


In [0]:
print(dbutils.fs.head("/FileStore/orders_csv.csv",1000))

[Truncated to first 1000 bytes]
order_id,order_date,order_customer_id,order_status
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE
6,2013-07-25 00:00:00.0,7130,COMPLETE
7,2013-07-25 00:00:00.0,4530,COMPLETE
8,2013-07-25 00:00:00.0,2911,PROCESSING
9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT
10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT
11,2013-07-25 00:00:00.0,918,PAYMENT_REVIEW
12,2013-07-25 00:00:00.0,1837,CLOSED
13,2013-07-25 00:00:00.0,9149,PENDING_PAYMENT
14,2013-07-25 00:00:00.0,9842,PROCESSING
15,2013-07-25 00:00:00.0,2568,COMPLETE
16,2013-07-25 00:00:00.0,7276,PENDING_PAYMENT
17,2013-07-25 00:00:00.0,2667,COMPLETE
18,2013-07-25 00:00:00.0,1205,CLOSED
19,2013-07-25 00:00:00.0,9488,PENDING_PAYMENT
20,2013-07-25 00:00:00.0,9198,PROCESSING
21,2013-07-25 00:00:00.0,2711,PENDING
22,2013-07-25 00:00:00.0,333,COMPLETE
23,2013-07-25 00:00:00.

In [0]:
%sql
COPY INTO orders
FROM 'dbfs:/FileStore/orders_csv.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')


num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
68881,68881,0


In [0]:
%sql

select count(*) from orders

count(1)
68881


In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/retail-data")
dbutils.fs.mkdirs("dbfs:/FileStore/retail-data/orders_data")
dbutils.fs.mkdirs("dbfs:/FileStore/retail-data/orders_checkpoint")

True

In [0]:
landing_zone = "dbfs:/FileStore/retail-data"
orders_data = landing_zone + "/orders_data"
checkpoint_path = landing_zone + "/orders_checkpoint"

In [0]:
orders_df = spark.readStream \
    .format("cloudfiles") \
        .option("cloudfiles.format","csv") \
            .option("cloudfiles.inferschema",'True') \
                .option("cloudfiles.inferColumnTypes","True") \
                    .option("cloudfiles.SchemaLocation",checkpoint_path) \
                        .load(orders_data)

# Here you cannot start streaming without uploading the files

-  either you can manually define the schema
- otherewise make sure there is a file in the folder


In [0]:
orders_df.display()

In [0]:
%sql
describe detail orders

In [0]:
orders_df.writeStream \
    .format("delta") \
        .option("checkpointLocation",checkpoint_path) \
            .outputMode("append") \
                .toTable("ordersdelta")


In [0]:
%sql

show tables

database,tableName,isTemporary
default,orders,false
default,ordersdelta,false


In [0]:
%sql
select count(*) from ordersdelta

count(1)
400


In [0]:
%sql
select * from ordersdelta

OrderID,OrderDate,CustomerID,TotalAmount,Status,_rescued_data
1,2023-01-01,25,6202.83,Cancelled,null
2,2023-01-02,32,9871.87,Cancelled,null
3,2023-01-03,78,6814.71,Completed,null
4,2023-01-04,89,4972.72,Pending,null
5,2023-01-05,91,5150.9,Pending,null
6,2023-01-06,6,6603.76,Pending,null
7,2023-01-07,20,7080.93,Cancelled,null
8,2023-01-08,15,4101.39,Cancelled,null
9,2023-01-09,16,1755.19,Pending,null
10,2023-01-10,34,4953.16,Pending,null


In [0]:
orders_schema = StructType([
    StructField("oder_id",IntegerType()),
    StructField("orderDate",TimestampType()),
    StructField("Customer_id",Integertype()),
    StructField("orderStatus",Stringtype())
])

In [0]:
orders_df.display(000)

In [0]:
oders_df = Spark.readStream \
    .format("cloudfiles") \
        .option("cloudFiles.format","csv") \
            .option("cloudFiles.inferSchema","true") \
                .option("cloudFiles.infercolumnTypes","true") \
                    .option("cloudfiles.SchemaLocation",checkpoint_path) \
                        .option("header","true") \
                            .load(orders_data)



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2888020156938058>, line 1
----> 1 oders_df = Spark.readStream \
      2     .format("cloudfiles") \
      3         .option("cloudFiles.format","csv") \
      4             .option("cloudFiles.inferSchema","true") \
      5                 .option("cloudFiles.infercolumnTypes","true") \
      6                     .option("cloudfiles.SchemaLocation",checkpoint_path) \
      7                         .option("header","true") \
      8                             .load(orders_data)

NameError: name 'Spark' is not defined

### OR

you can also use the below way to provide the schema hints for  certain coulmns alone if you want

In [0]:
oders_df = Spark.readStream \
    .format("cloudfiles") \
        .option("cloudFiles.format","csv") \
            .option("cloudFiles.inferSchema","true") \
              .option("cloudFiles.schemaHints","order_id",int )
                .option("cloudFiles.infercolumnTypes","true") \
                    .option("cloudfiles.SchemaLocation",checkpoint_path) \
                        .option("header","true") \
                            .load(orders_data)

